# Global Anomalies detection

Which config is going to be loaded ? normal or with extended variable

In [1]:
extendend_dataset_selected = False

### Libraries

In [2]:
import h2o
import pandas as pd
import numpy as np

In [3]:
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode() 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [4]:
import itertools

In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [6]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [7]:
from scipy.special import erf

In [8]:
from scipy.stats import pearsonr, kendalltau, spearmanr

In [9]:
if extendend_dataset_selected:
    from extendedgenv import *
else:
    from genv import *

Considered features for outliers

In [10]:
X_fraud

['V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'amount']

### H2O cluster

API doc : http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/index.html

In [11]:
h2o.init(max_mem_size = "32g")

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,31 mins 31 secs
H2O cluster version:,3.10.4.8
H2O cluster version age:,5 days
H2O cluster name:,H2O_from_python_pierre_2e44fe
H2O cluster total nodes:,1
H2O cluster free memory:,28.00 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [12]:
a = pd.read_csv(DIR_INPUT + FILE_EVALTEST)

In [13]:
a.isnull().values.any()

False

### Load data

Load data into h2oframe

In [14]:
df_train_unbalanced = h2o.import_file(path=DIR_INPUT + FILE_TRAIN_UNBALANCED, col_names=COL_NAME, col_types=COL_TYPE,header=1)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
df_validtest = h2o.import_file(path=DIR_INPUT + FILE_EVALTEST, col_names=COL_NAME, col_types=COL_TYPE, header=1)

Parse progress: |█████████████████████████████████████████████████████████| 100%


### Exploration 

In [16]:
df_train_unbalanced.head(1)

datetime,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,amount,fraud
0,-1.35981,-0.0727812,2.53635,1.37816,-0.338321,0.462388,0.239599,0.0986979,0.363787,0.0907942,-0.5516,-0.617801,-0.99139,-0.311169,1.46818,-0.470401,0.207971,0.0257906,0.403993,0.251412,-0.0183068,0.277838,-0.110474,0.0669281,0.128539,-0.189115,0.133558,-0.0210531,149.62,0


In [17]:
df_validtest.head(1)

datetime,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,amount,fraud
126397,-0.765586,1.11071,0.497975,-1.24207,0.809253,0.299673,0.352753,0.332642,0.00719612,-1.36338,0.00152343,0.759458,1.21781,-1.42397,-0.316525,1.14988,-0.371549,1.22995,0.55664,-0.0120937,-0.223609,-0.553292,-0.268529,-1.48553,-0.149443,-0.252863,-0.286616,-0.148096,3.79,0


# Tools

Collection of diverse function to be used afterwards

Percentage of common observation in outliers from 2 models

In [18]:
def index_outlier_common(df, outlier_column1, outlier_column2, quantile):
    """
    """
    assert(isinstance(quantile,float))
    assert(isinstance(outlier_column1,str))
    assert(isinstance(df,h2o.H2OFrame))
    dfo1 = df[outlier_column1].as_data_frame()
    dfo2 = df[outlier_column2].as_data_frame()
    df_percentile1 = float(dfo1.quantile(quantile))
    df_outlier1 = dfo1[(dfo1[outlier_column1] > df_percentile1)]
    index_list1 = df_outlier1.index.values
    df_percentile2 = float(dfo2.quantile(quantile))
    df_outlier2 = dfo2[(dfo2[outlier_column2] > df_percentile2)]
    index_list2 = df_outlier2.index.values
    max_common = min(len(index_list1), len(index_list2))
    common = len(list(set(index_list1).intersection(index_list2)))
    print("[{} <-> {}] {} common element in outliers out of {} ({}%)".format(outlier_column1, outlier_column2, common, max_common, common*100/max_common ))

Percentage of frauds in quantile

In [19]:
def fraud_in_outlier_quantile(df, outlier_column, quantile):
    """
    """
    print("WARNING : method run on frauds and not on cards")
    assert(isinstance(quantile,float))
    assert(isinstance(outlier_column,str))
    assert(isinstance(df,h2o.H2OFrame))
    df_percentile = float((df[outlier_column].as_data_frame()).quantile(quantile))
    fraud_percent = float(df[df['fraud']]['fraud'].dim[0] ) / df['fraud'].dim[0] *100
    df_outlier = df[(df[outlier_column] > df_percentile)]
    fraud_outlier_percent = float(df_outlier[df_outlier['fraud']]['fraud'].dim[0] ) / df_outlier['fraud'].dim[0] *100
    print("> {}% of fraud in outlier while {}% in all data ({} observations considered)".format(fraud_outlier_percent, fraud_percent, df_outlier.dim[0]))
    return fraud_outlier_percent

Percentage of frauds in top

In [92]:
def fraud_in_outlier_top(df, outlier_column, top):
    """
    Percentage of fraud for the method named outlier_column (PCA, NN, ...) in top n observations.
    """
    assert(isinstance(top,int))
    assert(isinstance(outlier_column,str))
    assert(isinstance(df,h2o.H2OFrame))
    fraud_percent = float(df[df['fraud']]['fraud'].dim[0] ) / df['fraud'].dim[0] *100
    df = df.as_data_frame().sort_values(outlier_column, ascending=False)
    fraud_outlier_percent = (df.head(top).copy().fraud.sum() / top) *100
    print("> {}% of fraud in outlier while {}% in all data ({} observations considered (not normalized!))".format(fraud_outlier_percent, fraud_percent, top))
    return fraud_outlier_percent

# Auto-encoder

Here the reconstruction error of the replicator neural network is used as outlier score

We will used a decreasing number of hidden layer at each step with a minimum for the central hidden layer

Example of using an autoencoder from h2o doc : https://github.com/nits555/h2o-3/blob/43dd8d37e990e2a3311613b66ee8d2ab65695184/h2o-docs/src/booklets/v2_2015/source/DeepLearning_Vignette_code_examples/deeplearning_anomaly.py

#### Structure definition

In [21]:
hd1 = len(X_autoenc) - 2
hd3 = len(X_autoenc) - 2
hidden_structure = [hd1, hd3]

In [22]:
X_autoenc

['V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'amount']

#### Learning

In [23]:
autoenc = h2o.estimators.deeplearning.H2OAutoEncoderEstimator(hidden=hidden_structure, epochs=200, activation='tanh', autoencoder=True)

In [24]:
autoenc.train(x                = X_autoenc, 
              training_frame   = df_train_unbalanced) # We learn on unbalanced dataset in order to define a 'normal' behavor

deeplearning Model Build progress: |██████████████████████████████████████| 100%


#### Learning result

In [25]:
print(autoenc)

Model Details
H2OAutoEncoderEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1495800034122_6


ModelMetricsAutoEncoder: deeplearning
** Reported on train data. **

MSE: 1.4478672525724291e-05
RMSE: 0.0038050850878428846
Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_mse
,2017-05-26 14:32:09,0.084 sec,0.00000 obs/sec,0.0,0,0.0,0.0632057,0.0039950
,2017-05-26 14:32:17,8.099 sec,25149 obs/sec,1.0828324,2,200115.0,0.0038051,0.0000145
,2017-05-26 14:32:26,16.759 sec,24107 obs/sec,2.1621962,4,399589.0,0.0041384,0.0000171
,2017-05-26 14:32:34,24.160 sec,25057 obs/sec,3.2452396,6,599743.0,0.0041310,0.0000171
,2017-05-26 14:32:42,32.320 sec,24995 obs/sec,4.3280071,8,799846.0,0.0039521,0.0000156
,2017-05-26 14:32:49,40.003 sec,25220 obs/sec,5.4083395,10,999499.0,0.0052469,0.0000275
,2017-05-26 14:32:57,47.717 sec,25484 obs/sec,6.4902412,12,1199442.0,0.0040000,0.0000160
,2017-05-26 14:33:05,55.543 sec,25531 obs/sec,7.5733062,14,1399600.0,0.0043262,0.0000187
,2017-05-26 14:33:15,1 min 5.987 sec,24568 obs/sec,8.6564849,16,1599779.0,0.0087941,0.0000773
,2017-05-26 14:33:21,1 min 12.048 sec,25316 obs/sec,9.7401235,18,1800043.0,0.0059584,0.0000355


#### Anomalies detection

Anomaly return the reconstruction error : the mean square error

In [26]:
autoenc_outlier_train = autoenc.anomaly(df_train_unbalanced[X_autoenc])

In [27]:
autoenc_outlier_validtest = autoenc.anomaly(df_validtest[X_autoenc])

In [28]:
df_train_unbalanced["outlier_autoenc"] = autoenc_outlier_train
df_validtest["outlier_autoenc"] = autoenc_outlier_validtest

# PCA (sklearn)

In [30]:
pca_data = df_train_unbalanced[X_pca].as_data_frame() #.dropna()
scaler = StandardScaler()
pca_data = scaler.fit_transform(pca_data)

In [31]:
pca_data[0]

array([-0.65303302, -0.05669755,  1.50034241,  0.92684611, -0.14342964,
        0.31360358,  0.25192171,  0.06095137,  0.3062364 ,  0.10034654,
       -0.67960464, -0.47799813, -1.00120486, -0.38651324,  1.43672991,
       -0.52492232,  0.1934398 ,  0.08323435,  0.51435717,  0.30658163,
        0.00261813,  0.50383168, -0.15299291,  0.10038666,  0.10980927,
       -0.40565372,  0.33545488, -0.07517851,  0.24604144])

In [32]:
pca = PCA(n_components=K_COMPONENTS)
pcafit = pca.fit_transform(pca_data)

In [33]:
pca.explained_variance_ratio_

array([ 0.0691448 ,  0.04965898,  0.04665676,  0.04268248,  0.04141516,
        0.03922256,  0.03768776,  0.03694641,  0.03622515,  0.03600767,
        0.03561654,  0.03540865,  0.03496107,  0.03460562,  0.03436162,
        0.03382436,  0.03357227,  0.03285833,  0.03264493,  0.03198605,
        0.03142355,  0.03116115,  0.03034153,  0.02971687,  0.02703233,
        0.02661704])

In [34]:
pca.explained_variance_ratio_.sum()

0.95177965108903972

In [35]:
pcareconstruct = pca.inverse_transform(pcafit)

In [36]:
outlier_pca = pd.DataFrame(np.mean(np.square(pcareconstruct - pca_data), axis=1))
outlier_pca.columns = ["outlier_pca"]                           
df_train_unbalanced["outlier_pca"] = h2o.H2OFrame(outlier_pca)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [37]:
pca_data = df_validtest[X_pca].as_data_frame()
pca_data = scaler.transform(pca_data)

In [38]:
pcafit = pca.transform(pca_data)

In [39]:
pcareconstruct = pca.inverse_transform(pcafit)

In [40]:
outlier_pca = pd.DataFrame(np.mean(np.square(pcareconstruct - pca_data), axis=1))
outlier_pca.columns = ["outlier_pca"]                           
df_validtest["outlier_pca"] = h2o.H2OFrame(outlier_pca)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [41]:
## if you run out of memory, do an explicit garbage collection at this point
#del pcareconstruct, outlier_pca, pca_data, pcafit

# Z-score

In [42]:
zscore_data = df_train_unbalanced[X_zscore].as_data_frame() # TODO X_pca
scaler = StandardScaler()
zscore_data = scaler.fit_transform(zscore_data)

In [43]:
zscorefit = pca.fit_transform(zscore_data)

In [44]:
outlier_zscore = pd.DataFrame(np.mean(np.square(zscorefit), axis=1))
outlier_zscore.columns = ["outlier_zscore"]                           
df_train_unbalanced["outlier_zscore"] = h2o.H2OFrame(outlier_zscore)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [45]:
zscore_data_valid = df_validtest[X_zscore].as_data_frame() # TODO X_pca
zscoretrans_valid = scaler.transform(zscore_data_valid)

In [46]:
outlier_zscore = pd.DataFrame(np.mean(np.square(zscoretrans_valid), axis=1))
outlier_zscore.columns = ["outlier_zscore"]                           
df_validtest["outlier_zscore"] = h2o.H2OFrame(outlier_zscore)

Parse progress: |█████████████████████████████████████████████████████████| 100%


# Ensemble

In [47]:
gaussian_norm = True

In [48]:
def gaussian_norm(series):
    mu = series.mean()
    std = series.std()
    series = np.array(series)
    series = (series - mu) / (std * np.sqrt(2))
    series = erf(series)
    series = [max(0,s) for s in series]
    return series

In [49]:
ensemble_df_train = df_train_unbalanced[OUTLIER_COLUMNS].as_data_frame()

In [50]:
if gaussian_norm:
    ensemble_df_train = ensemble_df_train.apply(gaussian_norm)
else:
    scaler_ensemble_train = MinMaxScaler()
    ensemble_df_train = scaler_ensemble_train.fit_transform(ensemble_df_train)

In [51]:
ensemble_df_train_true = ensemble_df_train[df_train_unbalanced[LABEL].as_data_frame()[LABEL] == True]
for col in OUTLIER_COLUMNS:
    data = [go.Histogram(x=ensemble_df_train_true[col], histnorm='probability')]
    layout = go.Layout(title='Frauds distribution across ' + col +' outlier score')
    fig = dict(data=data, layout=layout)
    plotly.offline.iplot(fig)

In [52]:
ensemble_train_res = np.sum(np.array(ensemble_df_train), axis=1) / len(OUTLIER_COLUMNS)

In [53]:
ensemble_train_res_true = pd.Series(ensemble_train_res)[df_train_unbalanced[LABEL].as_data_frame()[LABEL] == True]
data = [go.Histogram(x=ensemble_df_train_true[col], histnorm='probability')]
layout = go.Layout(title='Frauds distribution across ensemble outlier score')
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig)

In [54]:
outlier_ensemble_train = pd.DataFrame(ensemble_train_res)
outlier_ensemble_train.columns = ["ensemble_outlier"]                           
df_train_unbalanced["ensemble_outlier"] = h2o.H2OFrame(outlier_ensemble_train)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [55]:
ensemble_df = df_validtest[OUTLIER_COLUMNS].as_data_frame()

In [56]:
if gaussian_norm:
    ensemble_df = ensemble_df.apply(gaussian_norm)
else:
    scaler_ensemble = MinMaxScaler()
    ensemble_df = scaler_ensemble.fit_transform(ensemble_df)

In [57]:
ensemble_res = np.sum(np.array(ensemble_df), axis=1) / len(OUTLIER_COLUMNS)

In [58]:
outlier_ensemble = pd.DataFrame(ensemble_res)
outlier_ensemble.columns = ["ensemble_outlier"]                           
df_validtest["ensemble_outlier"] = h2o.H2OFrame(outlier_ensemble)

Parse progress: |█████████████████████████████████████████████████████████| 100%


# Save result

In [59]:
outname = DIR_OUTPUT + 'train_unbalanced_outlier.csv'
h2o.export_file(df_train_unbalanced, outname, force = True, parts = 1)

Export File progress: |███████████████████████████████████████████████████| 100%


In [60]:
outname = DIR_OUTPUT + 'evaltest_outlier.csv'
h2o.export_file(df_validtest, outname, force = True, parts = 1)

Export File progress: |███████████████████████████████████████████████████| 100%


# Evaluation

## Supervised evaluation

### Auto-encoder outlier eval

In [93]:
fraud_in_outlier_top(df_validtest, "outlier_autoenc", 1000)

> 9.6% of fraud in outlier while 0.124% in all data (1000 observations considered (not normalized!))


9.5999999999999996

### PCA outlier eval (in the context of fraud detection, in respect to their label)

In [94]:
fraud_in_outlier_top(df_validtest, "outlier_pca", 1000)

> 0.1% of fraud in outlier while 0.124% in all data (1000 observations considered (not normalized!))


0.10000000000000001

### Zscore outlier eval (in the context of fraud detection, in respect to their label)

In [95]:
fraud_in_outlier_top(df_validtest, "outlier_zscore", 1000)

> 5.2% of fraud in outlier while 0.124% in all data (1000 observations considered (not normalized!))


5.2000000000000002

### Outlier ensemble eval (in the context of fraud detection, in respect to their label)

In [96]:
fraud_in_outlier_top(df_validtest, "ensemble_outlier", 1000)

> 3.6999999999999997% of fraud in outlier while 0.124% in all data (1000 observations considered (not normalized!))


3.6999999999999997

## Correlation

In [62]:
outlierdf = df_validtest[OUTLIER_COLUMNS + OUTLIER_ENSEMBLE_COLUMNS].as_data_frame()

In [63]:
outliercol_combi = list(itertools.combinations(outlierdf.columns, 2))
outliercol_combi

[('outlier_autoenc', 'outlier_pca'),
 ('outlier_autoenc', 'outlier_zscore'),
 ('outlier_autoenc', 'ensemble_outlier'),
 ('outlier_pca', 'outlier_zscore'),
 ('outlier_pca', 'ensemble_outlier'),
 ('outlier_zscore', 'ensemble_outlier')]

### Pearson coefficient between score

In [64]:
outlierdf.corr()

,outlier_autoenc,outlier_pca,outlier_zscore,ensemble_outlier
outlier_autoenc,1.000000,0.398419,0.921967,0.096145
outlier_pca,0.398419,1.000000,0.472815,0.670720
outlier_zscore,0.921967,0.472815,1.000000,0.275684
ensemble_outlier,0.096145,0.670720,0.275684,1.000000


ok but significant ?

In [65]:
for combi in outliercol_combi:
    colname1, colname2 = combi
    coeff, pvalue = pearsonr(outlierdf[colname1], outlierdf[colname2])
    print("Correlation between {} and {} is {} with a pvalue={}".format(colname1, colname2 , coeff,pvalue))

Correlation between outlier_autoenc and outlier_pca is 0.3984191925667174 with a pvalue=0.0
Correlation between outlier_autoenc and outlier_zscore is 0.9219667543813767 with a pvalue=0.0
Correlation between outlier_autoenc and ensemble_outlier is 0.09614532866854555 with a pvalue=5.751744958052358e-204
Correlation between outlier_pca and outlier_zscore is 0.4728152811754703 with a pvalue=0.0
Correlation between outlier_pca and ensemble_outlier is 0.670720074987734 with a pvalue=0.0
Correlation between outlier_zscore and ensemble_outlier is 0.27568369382629854 with a pvalue=0.0


yes, corr are significant

### Spearman rank between score

In [66]:
for combi in outliercol_combi:
    colname1, colname2 = combi
    coeff, pvalue = spearmanr(outlierdf[colname1], outlierdf[colname2])
    print("Correlation between {} and {} is {} with a pvalue={}".format(colname1, colname2 , coeff,pvalue))

Correlation between outlier_autoenc and outlier_pca is 0.11096276901327883 with a pvalue=2.16013258827061e-271
Correlation between outlier_autoenc and outlier_zscore is 0.6271644840321677 with a pvalue=0.0
Correlation between outlier_autoenc and ensemble_outlier is 0.30488638815553937 with a pvalue=0.0
Correlation between outlier_pca and outlier_zscore is 0.18787448922830832 with a pvalue=0.0
Correlation between outlier_pca and ensemble_outlier is 0.7784134561328822 with a pvalue=0.0
Correlation between outlier_zscore and ensemble_outlier is 0.46286136085733826 with a pvalue=0.0


### Kendall's tau between score

In [67]:
for combi in outliercol_combi:
    colname1, colname2 = combi
    coeff, pvalue = kendalltau(outlierdf[colname1], outlierdf[colname2])
    print("Correlation between {} and {} is {} with a pvalue={}".format(colname1, colname2 , coeff,pvalue))

Correlation between outlier_autoenc and outlier_pca is 0.07306481979601237 with a pvalue=3.503357813526684e-263
Correlation between outlier_autoenc and outlier_zscore is 0.45319894352972306 with a pvalue=0.0
Correlation between outlier_autoenc and ensemble_outlier is 0.22434585759782016 with a pvalue=0.0
Correlation between outlier_pca and outlier_zscore is 0.13012462711831918 with a pvalue=0.0
Correlation between outlier_pca and ensemble_outlier is 0.6704419410042625 with a pvalue=0.0
Correlation between outlier_zscore and ensemble_outlier is 0.3452999120661913 with a pvalue=0.0


## Score comparison

#### Let's look at the outlier themselves (va index)

In [69]:
for combi in outliercol_combi:
    colname1, colname2 = combi
    index_outlier_common(df_validtest, colname1, colname2, 0.99)

[outlier_autoenc <-> outlier_pca] 155 common element in outliers out of 1000 (15.5%)
[outlier_autoenc <-> outlier_zscore] 598 common element in outliers out of 1000 (59.8%)
[outlier_autoenc <-> ensemble_outlier] 573 common element in outliers out of 1000 (57.3%)
[outlier_pca <-> outlier_zscore] 137 common element in outliers out of 1000 (13.7%)
[outlier_pca <-> ensemble_outlier] 249 common element in outliers out of 1000 (24.9%)
[outlier_zscore <-> ensemble_outlier] 619 common element in outliers out of 1000 (61.9%)


# Misc

In [98]:
h2o.cluster().show_status()

H2O cluster uptime:,49 mins 06 secs
H2O cluster version:,3.10.4.8
H2O cluster version age:,5 days
H2O cluster name:,H2O_from_python_pierre_2e44fe
H2O cluster total nodes:,1
H2O cluster free memory:,27.77 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [ ]:
stopbefore

In [ ]:
h2o.cluster().shutdown()